In [5]:
devtools::install_github("NPSDC/beaveR", ref = "dev")
setwd('/fs/cbcb-lab/rob/students/noor/Uncertainity/treeTerminusScripts')
suppressPackageStartupMessages(source("tree_helper_function.R"))
suppressPackageStartupMessages(library(beaveR))
clustFile <- "../brain_sim_nodtu_lowfc/mode=gc_bias/post_type=gibbs_nrep=100_tf=100/terminus/no_threshold_meaninf_0/group_nwk.txt"
quantDir <- file.path("../brain_sim_nodtu_lowfc/mode=gc_bias/post_type=gibbs_nrep=100_tf=100/out_sal") 
samples <- as.vector(outer(c(1:6), c(1,2), function(x,y) paste(x,y,sep='_')))
quantFiles <- file.path(quantDir, samples, 'quant.sf')
coldata <- data.frame(files=quantFiles, names=samples, condition=factor(rep(1:2, each=6)))
# tse <- buildTSE(treeTermFile = clustFile, coldata = coldata)

Skipping install of 'beaveR' from a github remote, the SHA1 (55f74bc0) has not changed since last install.
  Use `force = TRUE` to force installation



In [6]:
gc()

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,10084784,538.6,17397597,929.2,11659925,622.8
Vcells,17182504,131.1,26637336,203.3,22094052,168.6


In [31]:
saveDir <- "environment/brain_sim_nodtu_lowfc/mode=gc_bias/post_type=gibbs_nrep=100_tf=100/comp_trees/DE"
# save(tse, file=file.path(saveDir, "tseMean.RData"))
load(file=file.path(saveDir, "tseMean.RData"))

In [32]:
temp <- tse

In [9]:
load("environment/brain_sim_nodtu_lowfc/simulate.rda")
suppressPackageStartupMessages(library(TreeSummarizedExperiment))
suppressPackageStartupMessages(library(ape))
tree <- rowTree(tse)
l <- length(tree$tip)
missingTxps <- setdiff(rownames(tse)[1:l], rownames(sim.counts.mat))
dts <- Reduce(union, c(names(iso.dge)[iso.dge], names(iso.dte)[iso.dte], names(iso.dte.only)[iso.dte.only], 
                       names(iso.dtu)[iso.dtu]))
dts <- intersect(dts, rownames(sim.counts.mat))
sim.counts.mat <- rbind(sim.counts.mat, matrix(0, nrow = length(missingTxps), ncol = ncol(sim.counts.mat),
                                               dimnames = list(missingTxps, colnames(sim.counts.mat))))
sim.counts.mat <- sim.counts.mat[rownames(tse)[1:l],]
dts <- intersect(dts, rownames(sim.counts.mat))

innNodes <- l+1:tree$Nnode
aggCountsNodes <- computeAggNodesU(tree, c(1:length(tree$tip),innNodes), sim.counts.mat)
logFCNodes <- ifelse(rowSums(aggCountsNodes)==0, 0, log2(aggCountsNodes[,2]+1)-log2(aggCountsNodes[,1]+1))
names(logFCNodes)[(l+1):length(logFCNodes)] <- paste("Node", names(logFCNodes)[(l+1):length(logFCNodes)], sep="")
rootFC <- abs(logFCNodes[l+1])


In [10]:
length(dts)
summary(abs(logFCNodes[dts]))
print(mean(abs(logFCNodes[dts]) >= rootFC))
rootFC
mcols(tse)[["meanInfRV"]][l+1]

[1] 8778

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 0.4257  0.7805  1.0275  1.0097  1.2607  1.4836 

[1] 1


Node138894 
0.01270213

Node138894 
      0.01

#### Run Swish

In [11]:
set.seed(1)
y <- fishpond::scaleInfReps(temp[1:l,])
y <- labelKeep(y)
y <- swish(y, x="condition")

In [12]:
temp
mcols(y)

class: TreeSummarizedExperiment 
dim: 258923 12 
metadata(8): tximetaInfo quantInfo ... txpsAnn infRepsScaled
assays(105): counts abundance ... mean variance
rownames(258923): ENST00000484859.1 ENST00000601814.5 ... Node258922
  Node258923
rowData names(1): meanInfRV
colnames(12): 1_1 2_1 ... 5_2 6_2
colData names(2): names condition
reducedDimNames(0):
mainExpName: NULL
altExpNames(0):
rowLinks: a LinkDataFrame (258923 rows)
rowTree: 1 phylo tree(s) (138893 leaves)
colLinks: NULL
colTree: NULL

DataFrame with 138893 rows and 8 columns
                  meanInfRV log10mean      keep      stat     log2FC
                  <numeric> <numeric> <logical> <numeric>  <numeric>
ENST00000484859.1 0.0628568   1.69061      TRUE      6.13 -0.0498665
ENST00000601814.5 0.0692570   1.01785      TRUE     16.75  1.0137584
ENST00000419160.3 0.0629660   1.02768      TRUE     15.96  0.8947338
ENST00000416931.1 0.1089655   3.82013      TRUE     -1.23 -0.1046560
ENST00000457540.1 0.1041238   4.13343      TRUE      9.77  0.2921604
...                     ...       ...       ...       ...        ...
ENST00000583407.1  6.189095   2.09033      TRUE     -5.17 -0.2698960
ENST00000579002.5  1.475034   2.19953      TRUE     -1.38 -0.0765274
ENST00000581993.5  1.671231   1.46147      TRUE      3.14  0.2845811
ENST00000584678.1  1.417356   1.09895      TRUE      7.35  0.5233134
ENST00000579761.5  0.385673   1.14573      TRUE      0.12 -0.0426913
                       pvalue    locfdr    qvalue
            

In [13]:
detNodes <- list()
detNodes[["Txps"]] <- lapply(c(0.01, 0.05, 0.1), function(x) which(mcols(y)[,"qvalue"] <= x ))

negNodes <- list()
negNodes[["Txps"]] <- lapply(detNodes[["Txps"]], function(nodes) setdiff(seq(l), 
                                                          unlist(Descendants(tree, nodes))))

#### Run Genes

In [14]:
gse <- summarizeToGene(tse[1:l,])
gse <- fishpond::scaleInfReps(gse)
gse <- labelKeep(gse)
set.seed(1)
gse <- swish(gse, x="condition")
detNodes[["Genes"]] <- lapply(c(0.01, 0.05, 0.1), function(x) which(mcols(gse)[,"qvalue"] <= x ))
negNodes[["Genes"]] <- lapply(detNodes[["Genes"]], function(nodes) setdiff(seq(nrow(gse)), nodes))

loading existing TxDb created: 2022-08-19 22:12:59

Loading required package: GenomicFeatures

Loading required package: AnnotationDbi

obtaining transcript-to-gene mapping from database

loading existing gene ranges created: 2022-08-26 02:08:58

summarizing abundance

summarizing counts

summarizing length

summarizing inferential replicates



#### Min meanInfRV and Height

In [4]:
lambda <- c(0.05, 0.1, 1, 5, 10)

In [10]:
# cuts <- list()
# for(i in lambda) {
#     descSize <- sapply(Descendants(tree, seq(nrow(tse))), length)
#     metric <- (mcols(temp)[['meanInfRV']] + node.depth(tree, 2)*i) *descSize
#     objS <- solveForOptimalCut(tse, metVec = metric, type = 'min')
#     gammaN <- paste("gamma",i,sep="=")
#     cuts[[gammaN]] <- objS[["cut"]]
# }
names(cuts) <- paste("irv", "height", lambda, sep = "_") 
sapply(cuts,length)

irv_height_0.05  irv_height_0.1    irv_height_1    irv_height_5   irv_height_10 
          78314           89155          120488          134045          136828

In [11]:
save(cuts, file = file.path(saveDir, "cutsMean.RData"))

In [17]:
load(file = file.path(saveDir, "cutsMean.RData"))

In [18]:
for(i in seq_along(lambda)) {
     ihname <- paste("irv", "height", lambda[i], sep = "_") 
     y <- temp[sort(cuts[[ihname]]),]
     y <- fishpond::scaleInfReps(y)
     y <- labelKeep(y)
     set.seed(1)
     y <- swish(y, x="condition")
#      dim(y)
     detNodes[[ihname]] <- lapply(c(0.01, 0.05, 0.1), function(x) {
        nodes <- rownames(y)[which(mcols(y)[,"qvalue"] <= x)]
#         txps <- match(nodes[grep("ENST", nodes)], rownames(tse))
#         iNodes <- match(nodes[grep("Node", nodes)], rownames(tse))
         match(nodes, names(logFCNodes))
        
#         c(txps,iNodes)
    })
    negNodes[[ihname]] <- lapply(c(0.01, 0.05, 0.1), function(x) {
        nodes <- rownames(y)[setdiff(seq(nrow(y)), which(mcols(y)[,"qvalue"] <= x))]
        match(nodes, names(logFCNodes))
    })
}

In [19]:
names(detNodes)
names(negNodes)

[1] "Txps"            "Genes"           "irv_height_0.05" "irv_height_0.1" 
[5] "irv_height_1"    "irv_height_5"    "irv_height_10"

[1] "Txps"            "Genes"           "irv_height_0.05" "irv_height_0.1" 
[5] "irv_height_1"    "irv_height_5"    "irv_height_10"

#### Max Weighted Log Fold Change

In [25]:
getLog2FC <- function(tse, condition, pc=5, array=FALSE) {
  infRepIdx <- grep("infRep", assayNames(tse))
  infReps <- assays(tse)[infRepIdx]
  infRepsArray <- abind::abind(as.list(infReps), along=3)
  dims <- dim(infRepsArray)
    
  cond1 <- condition == levels(condition)[1]
  cond2 <- condition == levels(condition)[2]

  diffs <- matrix(nrow=dims[1],ncol=dims[3])
  for (k in seq_len(dims[3])) {
    diffs[,k] <- log2(rowMeans(infRepsArray[,cond2,k]) + pc) -
                 log2(rowMeans(infRepsArray[,cond1,k]) + pc)
  }
  if (array) {
    return(diffs)
  }
  # median over inferential replicates
  rowMedians(diffs)
}

In [26]:
tse <- computeSizeFactors(tse, type="txp")
tse <- scaleInfReps(tse)
lfc <- getLog2FC(tse, colData(tse)[["condition"]])
descSize <- sapply(phangorn::Descendants(tree, seq(nrow(tse))), length)
metric <- abs(lfc)/mcols(tse)[['meanInfRV']]
objS <- solveForOptimalCut(tse, metVec = metric, type = 'max')
cuts[["LFC"]] <- objS[["cut"]]

Setting inf rep depth to mean depth



Progress:   1 on 100Progress:   2 on 100Progress:   3 on 100Progress:   4 on 100Progress:   5 on 100Progress:   6 on 100Progress:   7 on 100Progress:   8 on 100Progress:   9 on 100Progress:  10 on 100Progress:  11 on 100Progress:  12 on 100Progress:  13 on 100Progress:  14 on 100Progress:  15 on 100Progress:  16 on 100Progress:  17 on 100Progress:  18 on 100Progress:  19 on 100Progress:  20 on 100Progress:  21 on 100Progress:  22 on 100Progress:  23 on 100Progress:  24 on 100Progress:  25 on 100Progress:  26 on 100Progress:  27 on 100Progress:  28 on 100Progress:  29 on 100Progress:  30 on 100Progress:  31 on 100Progress:  32 on 100Progress:  33 on 100Progress:  34 on 100Progress:  35 on 100Progress:  36 on 100Progress:  37 on 100Progress:  38 on 100Progress:  39 on 100Progress:  40 on 100Progress:  41 on 100Progress:  42 on 100Progress:  43 on 100Progress:  44 on 100Progress:  45 on 100Progress:  46 on 100Progress:  47 on 100Progress:  48 on 100Progress:  49 on 100Progress:  50 on 100

In [27]:
y <- temp[sort(cuts[["LFC"]]),]
y <- fishpond::scaleInfReps(y)
y <- labelKeep(y)
set.seed(1)
y <- swish(y, x="condition")

detNodes[["LFC"]] <- lapply(c(0.01, 0.05, 0.1), function(x) {
    nodes <- rownames(y)[which(mcols(y)[,"qvalue"] <= x)]
    match(nodes, names(logFCNodes))
})
negNodes[["LFC"]] <- lapply(c(0.01, 0.05, 0.1), function(x) {
        nodes <- rownames(y)[setdiff(seq(nrow(y)), which(mcols(y)[,"qvalue"] <= x))]
        match(nodes, names(logFCNodes))
})

#### TPM LFC Desc

In [28]:
lfcTPM <- getScaledLFC(temp, "condition")
metric <- abs(lfcTPM)/mcols(tse)[['meanInfRV']]*descSize
objS <- solveForOptimalCut(tse, metVec = metric, type = 'max')
cuts[["lfcTPMDesc"]] <- objS[["cut"]]
y <- temp[sort(cuts[["lfcTPMDesc"]]),]
y <- fishpond::scaleInfReps(y)
y <- labelKeep(y)
set.seed(1)
y <- swish(y, x="condition")

detNodes[["lfcTPMDesc"]] <- lapply(c(0.01, 0.05, 0.1), function(x) {
    nodes <- rownames(y)[which(mcols(y)[,"qvalue"] <= x)]
    match(nodes, names(logFCNodes))
})
negNodes[["lfcTPMDesc"]] <- lapply(c(0.01, 0.05, 0.1), function(x) {
        nodes <- rownames(y)[setdiff(seq(nrow(y)), which(mcols(y)[,"qvalue"] <= x))]
        match(nodes, names(logFCNodes))
    })

#### TPM LFC

In [33]:
# lfcTPM <- getScaledLFC(temp, "condition")
# metric <- abs(lfcTPM)/mcols(tse)[['meanInfRV']]
# objS <- solveForOptimalCut(tse, metVec = metric, type = 'max')
# cuts[["lfcTPM"]] <- objS[["cut"]]
y <- temp[sort(cuts[["lfcTPM"]]),]
y <- fishpond::scaleInfReps(y)
y <- labelKeep(y)
set.seed(1)
y <- swish(y, x="condition")

detNodes[["lfcTPM"]] <- lapply(c(0.01, 0.05, 0.1), function(x) {
    nodes <- rownames(y)[which(mcols(y)[,"qvalue"] <= x)]
    match(nodes, names(logFCNodes))
})
negNodes[["lfcTPM"]] <- lapply(c(0.01, 0.05, 0.1), function(x) {
        nodes <- rownames(y)[setdiff(seq(nrow(y)), which(mcols(y)[,"qvalue"] <= x))]
        match(nodes, names(logFCNodes))
    })

In [34]:
df <- data.frame(Method=character(), Metric=character(), FDR_0.01 = numeric(), FDR_0.05 = numeric(), FDR_0.10 = numeric())
for(n in names(detNodes))
{
    if(!n%in%c("term", "Genes")) {
        print(n)
        stats <-  sapply(seq_along(detNodes[[n]]), function(i) computeMetOut(detNodes[[n]][[i]], logFCNodes, tree = tree, negNodes = negNodes[[n]][[i]], lfcThresh = rootFC))
        colnames(stats) <- colnames(df)[c(3:5)]
        stats <- cbind(Method=rep(n,2), Metric = c("FDR", "TPR"), stats)
        df <- rbind(df, stats)
        
    }
    
}
print(df)

[1] "Txps"
[1] "tp 0.991852254209669"
[1] "tp 0.960747071858183"
[1] "tp 0.92046332046332"
[1] "irv_height_0.05"
[1] "tp 0.994401679496151"
[1] "tp 0.969097054563013"
[1] "tp 0.930008045052293"
[1] "irv_height_0.1"
[1] "tp 0.994659546061415"
[1] "tp 0.967406066093255"
[1] "tp 0.928517964071856"
[1] "irv_height_1"
[1] "tp 0.998225377107365"
[1] "tp 0.970499243570348"
[1] "tp 0.930379746835443"
[1] "irv_height_5"
[1] "tp 0.993513513513513"
[1] "tp 0.961702127659574"
[1] "tp 0.919787516600266"
[1] "irv_height_10"
[1] "tp 0.99906629318394"
[1] "tp 0.971338692764768"
[1] "tp 0.93026423317897"
[1] "LFC"
[1] "tp 0.991973244147157"
[1] "tp 0.963794683776352"
[1] "tp 0.917834153729648"
[1] "lfcTPMDesc"
[1] "tp 0.994772218073189"
[1] "tp 0.959728033472803"
[1] "tp 0.919133893062307"
[1] "lfcTPM"
[1] "tp 0.993931220498989"
[1] "tp 0.96228150873965"
[1] "tp 0.917620137299771"
              Method Metric     FDR_0.01   FDR_0.05   FDR_0.10
fdr             Txps    FDR  0.008147746 0.03925293 0.079536

#### Parse Terminus

In [35]:
suppressPackageStartupMessages(source("old_terminus_helper.R"))
dir <- "/fs/cbcb-lab/rob/students/noor/Uncertainity/brain_sim_nodtu_lowfc/mode=gc_bias/post_type=gibbs_nrep=100_tf=100/terminus/"
seBrainSim <- tximeta(coldata)
yTermThrNS <- seBrainSim[rownames(tse)[1:l],]
groupsClust <- parseClustFile(file.path(dir, "old", "1_1", "clusters.txt"), yTermThrNS)
mInds <- seq(nrow(yTermThrNS) + length(groupsClust))
yAggTermThrNS <- prepOSwish(yTermThrNS, mInds, groupsClust)

rownames(yAggTermThrNS)[l+1:length(groupsClust)] <- paste("Node", as.character(l+1:length(groupsClust)),sep="")
metadata(yAggTermThrNS)[["txpsAnn"]] <- rowData(yTermThrNS)
yAggTermThrNS <- yAggTermThrNS[-unlist(groupsClust),]
dim(yAggTermThrNS)

importing quantifications

reading in files with read_tsv

1 
2 
3 
4 
5 
6 
7 
8 
9 
10 
11 
12 


found matching transcriptome:
[ GENCODE - Homo sapiens - release 26 ]

loading existing TxDb created: 2022-08-19 22:12:59

loading existing transcript ranges created: 2022-08-19 22:13:01



[1] 130620     12

In [36]:
aggCountsNodes <- computeOAggNodesU(groupsClust, mInds, sim.counts.mat)
logFCNodesTerm <- ifelse(rowSums(aggCountsNodes)==0, 0, log2(aggCountsNodes[,2]+1)-log2(aggCountsNodes[,1]+1))
names(logFCNodesTerm)[(l+1):length(logFCNodesTerm)] <- paste("Node", names(logFCNodesTerm)[(l+1):length(logFCNodesTerm)], sep="")


In [37]:
yAggTermThrNS <- fishpond::scaleInfReps(yAggTermThrNS)
yAggTermThrNS <- labelKeep(yAggTermThrNS)
set.seed(1)
yAggTermThrNS <- swish(yAggTermThrNS, x="condition")

In [38]:
detNodes[["term"]] <- lapply(c(0.01, 0.05, 0.1), function(x) {
        nodes <- rownames(yAggTermThrNS)[which(mcols(yAggTermThrNS)[,"qvalue"] <= x)]
        match(nodes, names(logFCNodesTerm))
    })
negNodes[["term"]] <- lapply(c(0.01, 0.05, 0.1), function(x) {
        nodes <- rownames(yAggTermThrNS)[setdiff(seq(nrow(yAggTermThrNS)),
                                                 which(mcols(yAggTermThrNS)[,"qvalue"] <= x))]
        match(nodes, names(logFCNodesTerm))
    })
stats <-  sapply(seq_along(detNodes[["term"]]), function(i) computeMetOut(detNodes[["term"]][[i]], 
                                                                          logFCNodesTerm, tree = NULL, 
                                                                          negNodes = negNodes[["term"]][[i]],
                                                                          lfcThresh = rootFC))
colnames(stats) <- colnames(df)[c(3:5)]
stats <- cbind(Method=rep("term",2), Metric = c("FDR", "TPR"), stats)
df <- rbind(df, stats)


[1] "tp 0.992864983534577"
[1] "tp 0.961667787491594"
[1] "tp 0.920218579234973"


In [39]:
dg <- union(dge.genes, dte.genes)
length(dg)
sum(dg %in% rownames(gse))
dge.genes <- intersect(dge.genes, rownames(gse))
tG <- match(dg, rownames(gse))
length(tG)
stats <- sapply(detNodes[["Genes"]], function(genes) {
    computeTPFP(tG, genes, seq(nrow(gse)), seq(nrow(gse)))
})
colnames(stats) <- colnames(df)[c(3:5)]
stats <- cbind(Method=rep("Gene",2), Metric = c("FDR", "TPR"), stats)
df <- rbind(df, stats)

[1] 4122

[1] 4097

[1] 4122

In [40]:
suppressPackageStartupMessages(library(tidyr))
dfA <- df %>% pivot_wider(names_from = Metric, values_from = c(3:5))
dfA <- dfA[,c(1,seq(2,7,2),seq(3,7,2))]
colnames(dfA)[c(2:7)] <- as.vector(outer(c(0.01,0.05,0.1), c("FDR","TPR"), function(x,y) paste(x,y,sep="_")))
for(i in seq(ncol(dfA)-1))
    dfA[,i+1] <- round(as.numeric(unlist(dfA[,i+1])),3)
dfA

Method,0.01_FDR,0.05_FDR,0.1_FDR,0.01_TPR,0.05_TPR,0.1_TPR
<named list>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Txps,0.008,0.039,0.080,0.208,0.346,0.407
irv_height_0.05,0.006,0.031,0.070,0.280,0.395,0.455
irv_height_0.1,0.005,0.033,0.071,0.269,0.386,0.448
irv_height_1,0.002,0.030,0.070,0.152,0.347,0.417
irv_height_5,0.006,0.038,0.080,0.219,0.351,0.413
irv_height_10,0.001,0.029,0.070,0.124,0.323,0.397
LFC,0.008,0.036,0.082,0.283,0.401,0.463
lfcTPMDesc,0.005,0.040,0.081,0.331,0.456,0.516
lfcTPM,0.006,0.038,0.082,0.281,0.399,0.459


In [ ]:
sum(cuts$irv_height_0.05 > l)

In [ ]:
length(unlist(groupsClust))

In [41]:
saveDir <- "environment/brain_sim_nodtu_lowfc/mode=gc_bias/post_type=gibbs_nrep=100_tf=100/comp_trees/DE"
save(negNodes, file = file.path(saveDir, "negNodesMean.RData"))
save(detNodes, file = file.path(saveDir, "detNodesMean.RData"))
save(cuts, file=file.path(saveDir, "cutsMean.RData"))
save(df, file=file.path(saveDir, "dfMean.RData"))

In [ ]:
suppressPackageStartupMessages(library(tidyr))
suppressPackageStartupMessages(library(ggplot2))
suppressPackageStartupMessages(library(ggpubr))
load(file=file.path(saveDir, "dfCons.RData"))
dfB <- cbind(gather(df[seq(1, nrow(df), 2),], key = "NFDR", value = "FDR", 3:5),
gather(df[seq(2, nrow(df), 2),], key = "NFDR", value = "TPR", 3:5))
dfB <- dfB[, c(1,3,4,8)]
dfB[["meetCriteria"]] <- rep(FALSE, nrow(dfB))
l <- length(unique(df$Method))
for(i in seq(3)) {
    start <- (i-1)*l+1
    end <- start+l-1
    dfB$meetCriteria[start:end] <- dfB$FDR[start:end] <= c(0.01,0.05,0.1)[i]
}

dfB <- data.frame(do.call(cbind, lapply(dfB, unlist)))
dfB[,c(3)] <- as.numeric(dfB[,c(3)])
dfB[,c(4)] <- as.numeric(dfB[,c(4)])
dfB

In [ ]:
options(repr.plot.width=15, repr.plot.height=10)
size <- 20
pMean <- ggplot(dfB, aes(x = FDR, y = TPR, 
                             color = Method,
                             fill = ifelse(meetCriteria, Method, NA))) + 
                geom_point(size=5,shape=21) + 
                scale_fill_discrete(na.value = "transparent") +
                geom_line() +
                theme_bw() +
                geom_vline(xintercept = c(0.01, 0.05, 0.1), linetype = "dashed") +
                ggtitle("BrSimNorm Dataset") +
                theme(axis.title = element_text(size = size),
                plot.title = element_text(size = size + 1, hjust=0.5),
                axis.text = element_text(size = size),
                legend.text = element_text(size = size),
                legend.title = element_text(size = size),
                legend.position = "bottom") +
                guides(fill = "none")
pCons

In [42]:
negNodesDesc <- list()
for(n in names(negNodes)) {
    if(n == "Swish")
        negNodesDesc[[n]] <- negNodes[[n]]
    if(n != "term")
        negNodesDesc[[n]] <- lapply(negNodes[[n]], function(x) unlist(Descendants(tree, x)))
}
negNodesDesc[["term"]] <- lapply(negNodes[["term"]], function(x) {
    dd <- x[x>l]
    r <- setdiff(x, dd)
    dd <- dd - l
    c(unlist(groupsClust[dd]), r) 
})

In [ ]:
sapply(negNodes[["lfcTPM"]],length)
sapply(negNodesDesc[["lfcTPM"]],length)


In [43]:
dfO <- data.frame(Method=character(), Metric=character(), FDR_0.01 = numeric(), FDR_0.05 = numeric(), FDR_0.10 = numeric())
for(n in names(detNodes))
{
    print(n)
    if(!n %in% c("term", "Genes")) {
        stats <-  sapply(seq_along(detNodes[[n]]), function(i) computeMetOut(detNodes[[n]][[i]], 
                                                                             logFCNodes, tree = tree, 
                                                                             negNodes = negNodesDesc[[n]][[i]], 
                                                                             lfcThresh = rootFC))
        colnames(stats) <- colnames(df)[c(3:5)]
        stats <- cbind(Method=rep(n,2), Metric = c("FDR", "TPR"), stats)
        dfO <- rbind(dfO, stats)
    }
    
}
stats <-  sapply(seq_along(detNodes[["term"]]), function(i) computeMetOut(detNodes[["term"]][[i]], 
                                              logFCNodesTerm, tree = NULL, 
                                              negNodes = negNodesDesc[["term"]][[i]],
                                              lfcThresh = rootFC))
colnames(stats) <- colnames(df)[c(3:5)]
stats <- cbind(Method=rep("term",2), Metric = c("FDR", "TPR"), stats)
dfO <- rbind(dfO, stats)

tG <- match(dg, rownames(gse))
stats <- sapply(detNodes[["Genes"]], function(genes) {
    computeTPFP(tG, genes, seq(nrow(gse)), seq(nrow(gse)))
})
colnames(stats) <- colnames(df)[c(3:5)]
stats <- cbind(Method=rep("Genes",2), Metric = c("FDR", "TPR"), stats)
dfO <- rbind(dfO, stats)

print(dfO)

[1] "Txps"
[1] "tp 0.991852254209669"
[1] "tp 0.960747071858183"
[1] "tp 0.92046332046332"
[1] "Genes"
[1] "irv_height_0.05"
[1] "tp 0.994401679496151"
[1] "tp 0.969097054563013"
[1] "tp 0.930008045052293"
[1] "irv_height_0.1"
[1] "tp 0.994659546061415"
[1] "tp 0.967406066093255"
[1] "tp 0.928517964071856"
[1] "irv_height_1"
[1] "tp 0.998225377107365"
[1] "tp 0.970499243570348"
[1] "tp 0.930379746835443"
[1] "irv_height_5"
[1] "tp 0.993513513513513"
[1] "tp 0.961702127659574"
[1] "tp 0.919787516600266"
[1] "irv_height_10"
[1] "tp 0.99906629318394"
[1] "tp 0.971338692764768"
[1] "tp 0.93026423317897"
[1] "LFC"
[1] "tp 0.991973244147157"
[1] "tp 0.963794683776352"
[1] "tp 0.917834153729648"
[1] "lfcTPMDesc"
[1] "tp 0.994772218073189"
[1] "tp 0.959728033472803"
[1] "tp 0.919133893062307"
[1] "lfcTPM"
[1] "tp 0.993931220498989"
[1] "tp 0.96228150873965"
[1] "tp 0.917620137299771"
[1] "term"
[1] "tp 0.992864983534577"
[1] "tp 0.961667787491594"
[1] "tp 0.920218579234973"
               Meth

In [ ]:
dfO

In [44]:
save(dfO, file=file.path(saveDir, "dfConsO.RData"))

In [45]:
dfB <- dfO %>% pivot_wider(names_from = Metric, values_from = c(3:5))
dfB <- dfB[,c(1,seq(2,7,2),seq(3,7,2))]
colnames(dfB)[c(2:7)] <- as.vector(outer(c(0.01,0.05,0.1), c("FDR","TPR"), function(x,y) paste(x,y,sep="_")))
for(i in seq(ncol(dfB)-1))
    dfB[,i+1] <- round(as.numeric(unlist(dfB[,i+1])),3)
dfB

Method,0.01_FDR,0.05_FDR,0.1_FDR,0.01_TPR,0.05_TPR,0.1_TPR
<named list>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Txps,0.008,0.039,0.080,0.208,0.346,0.407
irv_height_0.05,0.006,0.031,0.070,0.229,0.341,0.402
irv_height_0.1,0.005,0.033,0.071,0.228,0.342,0.404
irv_height_1,0.002,0.030,0.070,0.140,0.333,0.403
irv_height_5,0.006,0.038,0.080,0.216,0.348,0.410
irv_height_10,0.001,0.029,0.070,0.123,0.321,0.395
LFC,0.008,0.036,0.082,0.239,0.358,0.421
lfcTPMDesc,0.005,0.040,0.081,0.240,0.364,0.428
lfcTPM,0.006,0.038,0.082,0.237,0.355,0.417


In [46]:
tse

class: TreeSummarizedExperiment 
dim: 258923 12 
metadata(8): tximetaInfo quantInfo ... txpsAnn infRepsScaled
assays(105): counts abundance ... mean variance
rownames(258923): ENST00000484859.1 ENST00000601814.5 ... Node258922
  Node258923
rowData names(1): meanInfRV
colnames(12): 1_1 2_1 ... 5_2 6_2
colData names(2): names condition
reducedDimNames(0):
mainExpName: NULL
altExpNames(0):
rowLinks: a LinkDataFrame (258923 rows)
rowTree: 1 phylo tree(s) (138893 leaves)
colLinks: NULL
colTree: NULL

In [ ]:
load(file=file.path(saveDir, "dfConsO.RData"))
dfB <- cbind(gather(dfO[seq(1, nrow(dfO), 2),], key = "NFDR", value = "FDR", 3:5),
gather(dfO[seq(2, nrow(dfO), 2),], key = "NFDR", value = "TPR", 3:5))
dfB <- dfB[, c(1,3,4,8)]
dfB[["meetCriteria"]] <- rep(FALSE, nrow(dfB))
l <- length(unique(df$Method))
for(i in seq(3)) {
    start <- (i-1)*l+1
    end <- start+l-1
    dfB$meetCriteria[start:end] <- dfB$FDR[start:end] <= c(0.01,0.05,0.1)[i]
}

dfB <- data.frame(do.call(cbind, lapply(dfB, unlist)))
dfB[,c(3)] <- as.numeric(dfB[,c(3)])
dfB[,c(4)] <- as.numeric(dfB[,c(4)])
dfB

In [ ]:
options(repr.plot.width=15, repr.plot.height=10)
size <- 20
pConsDesc <- ggplot(dfB, aes(x = FDR, y = TPR, 
                             color = Method,
                             fill = ifelse(meetCriteria, Method, NA))) + 
                geom_point(size=5,shape=21) + 
                scale_fill_discrete(na.value = "transparent") +
                geom_line() +
                theme_bw() +
                geom_vline(xintercept = c(0.01, 0.05, 0.1), linetype = "dashed") +
                ggtitle("BrSimNorm Dataset") +
                theme(axis.title = element_text(size = size),
                plot.title = element_text(size = size + 1, hjust=0.5),
                axis.text = element_text(size = size),
                legend.text = element_text(size = size),
                legend.title = element_text(size = size),
                legend.position = "bottom") +
                guides(fill = "none")
pConsDesc

In [ ]:
ggarrange(pCons, pConsDesc, common.legend = T, labels = c("Nodes", "Desc Leaves"), legend = "bottom")

In [ ]:
lambda <- c(0.1, 0.5, 1, 5)
gamma <- c(0.1,1,5)
for(i in lambda) {
    for(j in gamma) {
            name <- paste("irv", paste("height",i,sep="="), paste("LFC",j,sep="="), sep="_")
            print(name)
    }
    
}
